In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
exactness=pd.DataFrame([], columns=["batch_size","total probs","mpc same","percent same","precision"])
mean_errors=pd.DataFrame([], columns=["batch_size","mean RMSE","precision"])
RMSE={}
# exactness
for j,batch in enumerate([2,4,6,8,10,12]):
    gur=pd.read_csv('gurobi_results_2**'+str(batch)+'.csv',
                 header=None,names=np.arange(1,7,1).astype(str))
    mpc=pd.read_csv('mpc_results_2**'+str(batch)+'.csv',
                    header=None,names=np.arange(1,7,1).astype(str))
    for dec_places in [3,4]:
        equality=pd.DataFrame(gur.round(dec_places)==mpc.round(dec_places), columns=np.arange(1,7,1).astype(str))
        ind=np.repeat(np.array([True]),len(equality))
        for  i in np.arange(1,7,1).astype(str):
            ind=ind & equality[i] 
        exactness=exactness.append(pd.Series({"batch_size":"2**"+str(batch),"total probs":len(gur),
                                              "mpc same":len(ind[ind==True]),
                                              "percent same":str(np.round(len(ind[ind==True])*100/len(gur),1))+"%",
                                              "precision":dec_places}),ignore_index=True)
        
        er=((gur.round(dec_places)-mpc.round(dec_places))**2)
        er["pdmean"]=np.sqrt(er.mean(axis=1))
        er['rmse']=0
        for col in np.arange(1,7,1).astype(str):
            er['rmse']+=er[col]
        er["rmse"]=er["rmse"]/6
        er["rmse"]=np.sqrt(er["rmse"])
        RMSE[batch]=np.array(er["rmse"])
        mean_errors=mean_errors.append(pd.Series({"batch_size":"2**"+str(batch),
                                        "mean RMSE":np.mean(er["rmse"]),
                                        "precision":dec_places}),ignore_index=True)
exactness

,batch_size,total probs,mpc same,percent same,precision
0,2**2,4800,4507,93.9%,3
1,2**2,4800,3759,78.3%,4
2,2**4,19200,18456,96.1%,3
3,2**4,19200,15823,82.4%,4
4,2**6,76800,74584,97.1%,3
5,2**6,76800,65556,85.4%,4
6,2**8,307200,300316,97.8%,3
7,2**8,307200,268491,87.4%,4
8,2**10,12288,12127,98.7%,3
9,2**10,12288,10865,88.4%,4


In [3]:
mean_errors

,batch_size,mean RMSE,precision
0,2**2,0.000303,3
1,2**2,0.000299,4
2,2**4,0.000022,3
3,2**4,0.000020,4
4,2**6,0.000015,3
5,2**6,0.000013,4
6,2**8,0.000015,3
7,2**8,0.000014,4
8,2**10,0.000006,3
9,2**10,0.000006,4


In [7]:
batches=np.array(["2**4","2**6","2**8","2**10","2**12"])
fig = go.Figure(data=[
    go.Bar(name='mpc-solver', x=batches, y=exactness[exactness["precision"]==3]["percent same"],
           text=exactness[exactness["precision"]==3]["percent same"],textposition='outside',marker_color='steelblue',width=0.4),
])
# Change the bar mode
fig.update_yaxes(title="Accuracy (log scale)",type="log")
fig.update_xaxes(title="Batch Size")
fig.show()

In [9]:
batches=np.array(["2**4","2**6","2**8","2**10","2**12"])
fig = go.Figure(data=[
    go.Bar(name='Accurate to 4 decimal places', x=batches, y=exactness[exactness["precision"]==4]["percent same"], 
           text=exactness[exactness["precision"]==4]["percent same"],textposition='outside',marker_color='steelblue'),
    go.Bar(name='Accurate to 3 decimal places', x=batches, y=exactness[exactness["precision"]==3]["percent same"], 
           text=exactness[exactness["precision"]==3]["percent same"],textposition='outside',marker_color='darkcyan'),
#     go.Bar(name='Total Problems Solved', x=batches, y=exactness[exactness["precision"]==4]["total probs"],
#           marker_color='sienna')
])
# Change the bar mode
fig.update_yaxes(title="Accuracy (log scale)",type="log")
fig.update_xaxes(title="Batch Size")
fig.update_layout(barmode='group',title="Accuracy averaged over 100 runs for 1 epoch ")
fig.show()

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=batches, y=mean_errors[mean_errors["precision"]==4]["mean RMSE"],
                    mode='lines+markers',
                    name='4 decimal places'))
fig.add_trace(go.Scatter(x=batches, y=mean_errors[mean_errors["precision"]==3]["mean RMSE"],
                    mode='lines+markers',
                    name='3 decimal places'))
fig.update_yaxes(title="Average RMSE over all problems")
fig.update_xaxes(title="Batch Size")
fig.update_layout(title="Errors averaged over 100 runs ")
fig.show()

In [14]:
# for key in RMSE:
#     print(key)
#     print(RMSE[key])
# fig = go.Figure()
# for key in [2]:
#     n_probs=np.arange(0,len(RMSE[key]),1)
#     fig.add_trace(go.Scatter(x=n_probs, y=RMSE[key],
#                     mode='lines',
#                     name='2**'+str(key)))
# fig.show()
# fig = make_subplots(rows=1, cols=4, shared_yaxes=True)
# i=1
# for key in RMSE:
#     n_probs=np.arange(0,len(RMSE[key]),1)
#     fig.add_trace(go.Scatter(x=n_probs, y=RMSE[key],
#                     mode='lines',
#                     name='2**'+str(key)),row=1,col=i)
#     i+=1
# fig.update_yaxes(title="log(RMSE over all problems)",row=1,col=1)
# fig.update_yaxes(type="log")
# fig.update_layout(title="RMSE for all the problems solved")
# fig.show()